In [ ]:
# using GuSTO
include("../src/GuSTO.jl")

In [ ]:
robot = Astrobee3D()
model = AstrobeeSE3Manifold()
env = ISSCorner();
# add_obstacles!(env)

x_min_obs = [9.7, -0.1, 4.7]
x_max_obs = [9.8, 0.1, 4.8]
push!(env.obstacle_set, HyperRectangle(Vec3f0(x_max_obs), Vec3f0(x_max_obs-x_min_obs)))

# Small corner maneuver
# x_init = [9.2; 0.8; 4.2]
# x_goal = [10.9; 2.0; 5.0]

# Small free maneuver
# x_init = [9.2; 0.8; 4.2]
# x_goal = x_init + 1.*[1.; -0.25; 0.5]

# Straight free maneuver
x_init = [9.2; 0.; 4.8]
x_goal = x_init + [1.5; 0.; 0.]

# x_init = [10.2; -0.8; 5.6]
# x_goal = [10.2; 6.9; 4.2]

# x_init = [x_init; zeros(3); quat2mrp(sqrt(1/3)*[1.; 0.; 1.; 1.]); zeros(3)]
# x_goal = [x_goal; zeros(3); quat2mrp([-0.5; 0.5; -0.5; 0.5]); zeros(3)]
# x_init = [x_init; zeros(3); GuSTO.quat2mrp([0.; 0.; 0.; 1.]); zeros(3)]
# x_goal = [x_goal; zeros(3); GuSTO.quat2mrp([0.; 0.; 0.; -1.]); zeros(3)]
x_init = [x_init; zeros(3); [0.; 0.; 0.; 1.]; zeros(3)]
x_goal = [x_goal; zeros(3); [0.; 0.; 0.; 1.]; zeros(3)]

N = 30
tf_guess = 70.

PD = ProblemDefinition(robot, model, env, x_init, x_goal);
TOP = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOS_SCP = TrajectoryOptimizationSolution(TOP)

solve_SCP!(TOS_SCP, TOP, solve_gusto_jump!, init_traj_straightline, "Ipopt", print_level=0)

# solve_SCP!(TOSgusto, TOPgusto, solve_gusto_jump!, init_traj_nothing, "Ipopt", print_level=0)

In [ ]:
@show TOS_SCP.SCPS.converged
@show TOS_SCP.SCPS.iterations
@show TOS_SCP.SCPS.total_time
@show TOS_SCP.SCPS.accept_solution
@show TOS_SCP.SCPS.prob_status
@show TOS_SCP.SCPS.convergence_measure
@show TOS_SCP.SCPS.param.alg.ω_vec
@show TOS_SCP.SCPS.param.alg.Δ_vec
@show TOS_SCP.SCPS.J_true[end]
;

In [ ]:
using Plots
gr(fmt=:png)
plot([TOS_SCP.SCPS.traj.X[1,:]], [TOS_SCP.SCPS.traj.X[2,:]],
    xlabel = "x",
    ylabel = "y",
    aspect_ratio = 1,
    legend = :none)

In [ ]:
# for i in 1:29
#     @show TOS_SCP.SCPS.SCPC.nonconvex_state_convexified_ineq[:ncsi_obstacle_avoidance_signed_distance_convexified].con_reference[:,(27,)][i]
# end

In [ ]:
# for i in 1:29
#     @show TOS_SCP.SCPS.SCPC.nonconvex_state_convexified_ineq[:ncsi_obstacle_avoidance_potential].con_reference[:,(1,)][i]
# end

In [ ]:
TOS_SCPS  = TrajectoryOptimizationSolution(TOP)
solve_SCPshooting!(TOS_SCPS, TOP, solve_gusto_jump!, init_traj_straightline, "Ipopt", print_level=0)

In [ ]:
@show TOS_SCPS.SS.converged
@show TOS_SCPS.SS.prob_status
@show TOS_SCPS.SCPS.iterations
@show TOS_SCPS.total_time
@show TOS_SCPS.SS.iter_elapsed_times
@show TOS_SCPS.SS.J_true[end]

In [ ]:
using Plots
gr(fmt=:png)
plot([TOS_SCPS.SS.traj.X[1,:]], [TOS_SCPS.SS.traj.X[2,:]],
    xlabel = "x",
    ylabel = "y",
    aspect_ratio = 1,
    legend = :none)

In [ ]:
using AstrobeeRobot

# Animate Astrobee trajectory
vis = Visualizer()
delete!(vis)

vis[:goal]
for (idx,obs) in enumerate(env.keepout_zones)
    setobject!(vis[:goal][:goal], 
        Object(HyperSphere(Point3f0(x_goal[1:3]), 0.1f0),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))
end

vis[:workspace]
for (idx,ws) in enumerate(env.keepin_zones)
    if idx in (5,8)
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3), depthWrite=false)))
    else
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3))))
    end
end

vis[:obstacle]
for (idx,ws) in enumerate(env.obstacle_set)
    setobject!(vis[:workspace][Symbol(string("ws",idx+length(env.keepin_zones)))],
        Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.26,0.26,0.3))))
end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);

speed_factor = 3

Qs = Vector{Vector{Float64}}(0)
for k in 1:speed_factor:N
#     q = [quat_inv(TOS_SCP.SCPS.traj.X[7:10,k]); TOS_SCP.SCPS.traj.X[1:3,k]]
    q = [quat_inv(TOS_SCPS.SS.traj.X[7:10,k]); TOS_SCPS.SS.traj.X[1:3,k]]
    push!(Qs,q)
end

trans = Translation(14., -1., 7.)
rot = LinearMap(RotZ(-0.6)) ∘ LinearMap(RotY(-0.2))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.9)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

sleep(3)
setanimation!(mvis,1:length(Qs),Qs)

plot_in_cell = false
plot_in_cell ? IJuliaCell(vis) : open(vis)